In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
import pandas as pd
import re
import math
import time
import os, sys
import requests
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.action_chains import ActionChains


In [ ]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)


In [ ]:
def get_tch_home():
    num= 0
    tch_link=[]
    for i in driver.find_elements_by_xpath('//*[@id="container"]/div/div[2]/ul/li[*]'):
        num=num+1

        i.click()

        time.sleep(1)

        if num <5:
            for i in driver.find_elements_by_xpath('//*[@id="teacher_tab_cont"]/div[2]/ul/li[*]/ul/li[*]/div/a'):
                tch_link.append(i.get_attribute('href'))
        else:
            for i in driver.find_elements_by_xpath('//*[@id="teacher_tab_cont"]/div[2]/ul/li[*]/dl/dd[*]/a'):
                tch_link.append(i.get_attribute('href'))
                
    return(list(set(tch_link)))
        
    

In [ ]:
def get_tch_lec_link():
    tch_lec_link=[]
    for i in tch_link:
        tch_lec_link.append(re.sub('home?','lecList/',i))
    return(tch_lec_link)

In [ ]:
def get_lec_link():
    lec_link=[]
    for i in driver.find_elements_by_xpath('//*[@id="container"]/div[2]/div/div/div/ul/li[*]'):
        i.click()

        time.sleep(1)


        for j in driver.find_element_by_xpath('//*[@id="lectureLis"]').find_elements_by_class_name('summary-info'):
                lec_link.append(j.find_element_by_tag_name('a').get_attribute('href'))
                print(j.find_element_by_tag_name('a').text)

    return(lec_link)

In [ ]:
#선생님 이름

def get_tch_name():
    return(driver.find_element_by_class_name('name').text)

In [ ]:
#과목

def get_sub():
    return(driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[1]/td[1]').text)

In [ ]:
#선생님 id

def get_tch_id():
    try:
        value_1=re.split('=',driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[1]/div/div[1]/ul/li[2]').get_attribute('onclick'))[1]
        return(re.split('&',value_1)[0])
    except:
        return('선생님ID 없음')
    

In [ ]:
driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[1]/div/div[1]/ul/li[2]').get_attribute('onclick')

In [ ]:
#강좌명 

def get_lec_name():
    return(driver.find_element_by_class_name('tit-courses').text)

In [ ]:
#강의ID

def get_lec_id():
    return(re.sub('[^0-9]','',driver.current_url))

In [ ]:
#완강 여부

def get_complete():
    if re.findall('완강',driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td[2]').text)==['완강']:
        complete = 'O'
    else:
        complete = 'X'
    return(complete)

In [ ]:
#강의 정보 가져오기

def get_data_2():
    
    lec_num = int(re.sub('[^0-9]','',driver.find_element_by_xpath('//*[@id="container"]/div[1]/div[2]/div[2]/div/div[1]/div[2]/div[3]/div/h3/span').text))

    num=0
    data_2=[]
    list1=[]
    list2=[]
    list3=[]
    while True:

        for i in driver.find_elements_by_xpath('//*[@id="course_list_1"]/ol/li[*]/span[1]'):
            list1.append(i.text)


        for i in driver.find_elements_by_xpath('//*[@id="course_list_1"]/ol/li[*]/strong'):
            list2.append(i.text)


        for i in driver.find_elements_by_xpath('//*[@id="course_list_1"]/ol/li[*]/span[2]'):
            list3.append(i.text)

        num=num+1
        
        if lec_num//10==0:
            break
        
        try:
            driver.find_element_by_class_name('next').click()
        except:
            break

        time.sleep(0.3)


    list1=list(filter(('').__ne__, list1))
    list2=list(filter(('').__ne__, list2))
    list3=list(filter(('').__ne__, list3))
    data_2.append([','.join(['(' + str(a) + ', ' + b+', ' + c+')' for a,b,c in zip(list1,list2,list3)])])
    
    return(data_2)

In [ ]:
driver = open_chromedriver()

In [ ]:
driver.get('https://skyedu.conects.com/teachers/')

In [ ]:
time.sleep(3)

In [ ]:
tch_link = get_tch_home()

In [ ]:
tch_lec_link = get_tch_lec_link()

In [ ]:
num=0
lec_link=[]
for i in tch_lec_link:
    num=num+1
    print(num,'/',len(tch_lec_link))

    driver.get(i)

    time.sleep(1)

    lec_link.extend(get_lec_link())

    

In [ ]:
lec_link = list(set(lec_link))
lec_link = [item for item in lec_link if item != 'javascript:;' ]

In [ ]:
data_2=[]
num=0
tch_name =[]
sub = []
tch_id = []
lec_name =[]
lec_id = []
complete=[]

for i in lec_link:
    num=num+1
    driver.get(i)
    
    time.sleep(0.5)
    
    tch_name.append(get_tch_name())
    sub.append(get_sub())
    tch_id.append(get_tch_id())
    lec_name.append(get_lec_name())
    lec_id.append(get_lec_id())
    complete.append(get_complete())
    
    data_2.extend(get_data_2())
    
    print(num,'/',len(lec_link))

In [ ]:
data_2=[]
num=0
tch_name =[]
sub = []
tch_id = []
lec_name =[]
lec_id = []
complete=[]

for i in lec_link:
    num=num+1
    driver.get(i)
    
    time.sleep(0.5)
    
    tch_name.append(get_tch_name())
    sub.append(get_sub())
    tch_id.append(get_tch_id())
    lec_name.append(get_lec_name())
    lec_id.append(get_lec_id())
    complete.append(get_complete())
    
    data_2.extend(get_data_2())
    
    print(num,'/',len(lec_link))

In [ ]:
data = pd.DataFrame({'사이트':['스카이에듀']*len(sub),'과목':sub,'선생님':tch_name,'선생님ID':tch_id,'강좌명':lec_name,'강좌ID':lec_id,'완강여부':complete,'강의번호&챕터명&시간':data_2,'링크':lec_link})

In [ ]:
data

In [ ]:
writer = pd.ExcelWriter(r'스카이에듀.xlsx', engine='xlsxwriter', options={'strings_to_urls': True})
data.to_excel(writer)
writer.close()